<a href="https://colab.research.google.com/github/karoldem/travel-season/blob/main/travel_season.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
europe = "https://raw.githubusercontent.com/karoldem/travel-season/main/List_of_cities_by_average_temperature_1.csv"
europe = pd.read_csv(europe)



In [ ]:
import numpy, scipy.optimize

def fit_sin(tt, yy):
    '''Fit sin to the input time sequence, and return fitting parameters "amp", "omega", "phase", "offset", "freq", "period" and "fitfunc"'''
    tt = numpy.array(tt)
    yy = numpy.array(yy)
    ff = numpy.fft.fftfreq(len(tt), (tt[1]-tt[0]))   # assume uniform spacing
    Fyy = abs(numpy.fft.fft(yy))
    guess_freq = abs(ff[numpy.argmax(Fyy[1:])+1])   # excluding the zero frequency "peak", which is related to offset
    guess_amp = numpy.std(yy) * 2.**0.5
    guess_offset = numpy.mean(yy)
    guess = numpy.array([guess_amp, 2.*numpy.pi*guess_freq, 0., guess_offset])

    def sinfunc(t, A, w, p, c):  return A * numpy.sin(w*t + p) + c
    popt, pcov = scipy.optimize.curve_fit(sinfunc, tt, yy, p0=guess)
    A, w, p, c = popt
    f = w/(2.*numpy.pi)
    fitfunc = lambda t: A * numpy.sin(w*t + p) + c
    return {"amp": A, "omega": w, "phase": p, "offset": c, "freq": f, "period": 1./f, "fitfunc": fitfunc, "maxcov": numpy.max(pcov), "rawres": (guess,popt,pcov)}

In [1]:
import numpy, scipy.optimize

def fit_sin(tt, yy):
    '''Fit sin to the input time sequence, and return fitting parameters "amp", "omega", "offset" and "fitfunc"'''
    tt = numpy.array(tt)
    yy = numpy.array(yy)
    ff = numpy.fft.fftfreq(len(tt), (tt[1]-tt[0]))   # assume uniform spacing
    guess_amp = numpy.std(yy) * 2.**0.5
    guess_offset = numpy.mean(yy)
    guess = numpy.array([guess_amp, 0., guess_offset])

    def sinfunc(t, A, p, c):  return A * numpy.sin(0.5235987755982988*t + p) + c
    popt, pcov = scipy.optimize.curve_fit(sinfunc, tt, yy, p0=guess)
    A, p, c = popt
    f = 0.5235987755982988/(2.*numpy.pi)
    fitfunc = lambda t: A * numpy.sin(0.5235987755982988*t + p) + c
    return {"amp": A,  "phase": p, "offset": c, "fitfunc": fitfunc, "maxcov": numpy.max(pcov), "rawres": (guess,popt,pcov)}

In [7]:
from re import findall

def inject(data, dataframe, variable):
    for i in dataframe.index:

        k = []
        for j in ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']:
            try:
                string = dataframe.loc[i][j]
                string = string.replace('−', '-') #No, they are not the same
                string = findall(r"[-+]?(?:\d*\.*\d*)", string )
                string = list(   filter( ('').__ne__, string )   )[0]
                k.append ( float(string) )
                
            except:
                print(dataframe.loc[i])

        temporaryNvmVaiable = fit_sin([0,1,2,3,4,5,6,7,8,9,10,11], k)
        toAddtemporaryVariable = {}
        toAddtemporaryVariable[variable] = {}
        toAddtemporaryVariable[variable]['amplitude'] = temporaryNvmVaiable['amp']
        toAddtemporaryVariable[variable]['average'] = temporaryNvmVaiable['offset']
        toAddtemporaryVariable[variable]['phase'] = temporaryNvmVaiable['phase']

        data[(dataframe.loc[i]['City'], dataframe.loc[i]['Country'])] = toAddtemporaryVariable

In [8]:
import requests
import pandas as pd

data = {}

urls = [{'variable' : 'temperature', 'url' : 'https://en.wikipedia.org/wiki/List_of_cities_by_average_temperature'},
        {'variable' : 'sunshine', 'url' : 'https://en.wikipedia.org/wiki/List_of_cities_by_sunshine_duration'}]

for url in urls:
    html = requests.get(url['url']).content
    df_list = pd.read_html(html)
    for i in df_list:
            inject(data, i, url['variable'])



0                                                  NaN
1    This article uses bare URLs, which are uninfor...
Name: 0, dtype: object
0                                                  NaN
1    This article uses bare URLs, which are uninfor...
Name: 0, dtype: object
0                                                  NaN
1    This article uses bare URLs, which are uninfor...
Name: 0, dtype: object
0                                                  NaN
1    This article uses bare URLs, which are uninfor...
Name: 0, dtype: object
0                                                  NaN
1    This article uses bare URLs, which are uninfor...
Name: 0, dtype: object
0                                                  NaN
1    This article uses bare URLs, which are uninfor...
Name: 0, dtype: object
0                                                  NaN
1    This article uses bare URLs, which are uninfor...
Name: 0, dtype: object
0                                                  NaN
1    This arti

/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:262: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:254: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3440: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


ValueError: ignored

In [ ]:
data[('Warsaw', 'Poland')]

{'temperature': {'amplitude': 9.607631671680808,
  'average': 9.050000001191156,
  'phase': -1.576465373858028}}

In [ ]:
string = dataframe.loc[i][j]
string = string.replace('−', '-') #No, they are not the same
string = findall(r"[-+]?(?:\d*\.*\d*)", string )
string = list(   filter( ('').__ne__, string )   )[0]
k.append ( float(string) )

'−2.3'